In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath('..'))

from src.generators.lcg import LCG
from src.generators.mt_wrapper import MT19937
from src.generators.system_rng import SystemRNG
from src.generators.xor_hybrid import XORGenerator
from src.generators.bbs import BlumBlumShub

from src.analysis.test_suite import run_statistical_tests

In [2]:
# Configuration
SAMPLE_SIZE = 50000  # Taille de l'échantillon pour les tests

# Liste des générateurs à tester
generators = [
    ("LCG (Standard)", LCG(seed=42)),
    ("LCG (Faible/RANDU)", LCG(seed=42, a=65539, c=0, m=2**31)),
    ("Mersenne Twister", MT19937(seed=1234)),
    ("XOR Hybride", XORGenerator(seed=999)),
    ("System RNG (Crypto)", SystemRNG()),
    # BBS est lent, on réduit la taille d'échantillon pour lui ou on patiente
    ("Blum-Blum-Shub", BlumBlumShub(seed=42)) 
]

results_list = []

print(f"Lancement du benchmark sur {len(generators)} générateurs...")

for name, gen in generators:
    print(f"Testing {name}...", end=" ")
    
    # Pour BBS, on réduit un peu la taille si c'est trop lent (ex: 5000 bits)
    # Mais essayons avec la taille standard d'abord.
    current_size = SAMPLE_SIZE
    if "Blum" in name:
        current_size = 5000 # BBS est vraiment très lent (bit par bit)
        
    res = run_statistical_tests(gen, sample_size=current_size)
    
    # On aplatit le dictionnaire pour le DataFrame
    flat_res = {
        "Générateur": name,
        "Score Global": res["global_score"],
        "Entropie (bits)": res["entropy"]["value"],
        "Entropie Score": res["entropy"]["score"],
        "Chi-2 P-val": res["chi_squared"]["p_value"],
        "Chi-2 Verdict": res["chi_squared"]["verdict"],
        "Autocorr Score": res["autocorrelation"]["score"],
        "KS P-val": res["ks_test"]["p_value"]
    }
    results_list.append(flat_res)
    print("Done.")

print("Benchmark terminé.")

Lancement du benchmark sur 6 générateurs...
Testing LCG (Standard)... Done.
Testing LCG (Faible/RANDU)... Done.
Testing Mersenne Twister... Done.
Testing XOR Hybride... Done.
Testing System RNG (Crypto)... Done.
Testing Blum-Blum-Shub... Done.
Benchmark terminé.
